<a href="https://colab.research.google.com/github/MarcoCR1998/DM_Circulacion/blob/main/Optimizaci%C3%B3n_mediante_PuLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Código de Optimización de Distribución de Ejemplares por Sector - PuLP**

In [ ]:
!pip install pulp
!pip install pulp


In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

def optimizar_distribucion(ruta_archivo, total_ejemplares_por_dia, pronostico_ventas_por_dia):
    try:
        # Leer el archivo CSV
        df = pd.read_csv(ruta_archivo)

        # Crear el problema de optimización
        modelo = LpProblem("Optimización_Distribución", LpMaximize)

        # Crear variables de decisión para determinar promo, flete y envio_total por municpio y día de la semana
        envio_total = { (row.dia_semana, row.idmupiogeo): LpVariable(f"envio_{row.dia_semana}_{row.idmupiogeo}", lowBound=row.envio_total, cat='Integer')
                        for _, row in df.iterrows() }

        promo = { (row.dia_semana, row.idmupiogeo): LpVariable(f"promo_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, upBound=15, cat='Integer')
                  for _, row in df.iterrows() }

        flete = { (row.dia_semana, row.idmupiogeo): LpVariable(f"flete_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, upBound=30, cat='Integer')
                  for _, row in df.iterrows() }

        devuelto_opt = { (row.dia_semana, row.idmupiogeo): LpVariable(f"devuelto_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, cat='Integer')
                         for _, row in df.iterrows() }

        # Calcular un factor de devolución basado en datos históricos
        factor_devolucion = { (row.dia_semana, row.idmupiogeo): row.devuelto / row.envio_total if row.envio_total > 0 else 0
                              for _, row in df.iterrows() }

        # Función objetivo: Maximizar ventas y minimizar devoluciones
        modelo += lpSum((envio_total[i] - promo[i] - flete[i] - devuelto_opt[i]) for i in envio_total), "Maximizar_Ventas"

        # Restricción: No superar el total de ejemplares disponibles a nivel nacional por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] for i in envio_total if i[0] == dia) <= total_ejemplares_por_dia[dia], f"Restriccion_Total_Disponible_{dia}"

        # Restricción: La sumatoria de unidades vendidas no puede ser mayor al pronóstico de ventas por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] - promo[i] - flete[i] - devuelto_opt[i] for i in envio_total if i[0] == dia) <= pronostico_ventas_por_dia[dia], f"Restriccion_Pronostico_Ventas_{dia}"

        # Restricción: Asegurar que envio_total sea mayor o igual que vendido
        for (dia, municipio) in envio_total:
            modelo += envio_total[(dia, municipio)] >= df.set_index(['dia_semana', 'idmupiogeo']).loc[(dia, municipio), 'vendido'], f"Restriccion_Minimo_Envio_{dia}_{municipio}"

        # Restricción: Evitar valores negativos en Vendido_Optimizado
        for (dia, municipio) in envio_total:
            modelo += envio_total[(dia, municipio)] - promo[(dia, municipio)] - flete[(dia, municipio)] - devuelto_opt[(dia, municipio)] >= 0, f"Restriccion_No_Negativo_Ventas_{dia}_{municipio}"

        # Restricción: Ajustar Devuelto_Optimizado basado en Envio_Optimizado
        for (dia, municipio) in devuelto_opt:
            modelo += devuelto_opt[(dia, municipio)] == envio_total[(dia, municipio)] * factor_devolucion[(dia, municipio)], f"Restriccion_Devolucion_Segun_Envio_{dia}_{municipio}"

        # Resolver el problema
        modelo.solve()

        # Guardar los resultados en el DataFrame
        df["Envio_Optimizado"] = [envio_total[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Promo_Optimizado"] = [promo[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Flete_Optimizado"] = [flete[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Devuelto_Optimizado"] = [devuelto_opt[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Vendido_Optimizado"] = df["Envio_Optimizado"] - df["Promo_Optimizado"] - df["Flete_Optimizado"] - df["Devuelto_Optimizado"]

        # Guardar la solución en un CSV
        df.to_csv("solucion_optimizada_municipio.csv", index=False)
        print("Optimización completada. Resultados guardados en 'solucion_optimizada_municipio.csv'")

    except Exception as e:
        print(f"Error al ejecutar la optimización: {e}")

# Ruta del archivo CSV para sectores
ruta_csv = "/content/datos_finales_municipio.csv"  # Reemplaza con la ruta real

# Diccionarios con valores de total de ejemplares y pronóstico por día de la semana
total_ejemplares_por_dia = {
    "Lunes": 75000,
    "Martes": 75000,
    "Miercoles": 75000,
    "Jueves": 75000,
    "Viernes": 75000,
    "Sabado": 75000,
    "Domingo": 75000
}

pronostico_ventas_por_dia = {
    "Lunes": 68000,
    "Martes": 65000,
    "Miercoles": 61000,
    "Jueves": 68000,
    "Viernes": 63000,
    "Sabado": 63000,
    "Domingo": 69000
}

optimizar_distribucion(ruta_csv, total_ejemplares_por_dia, pronostico_ventas_por_dia)

**Pipeline: Optimización de ejemplares**

**Instalación de PuLP**
Antes de ejecutar el código, es necesario instalar la librería PuLP, que permite la resolución de problemas de optimización lineal y entera mixta.

In [ ]:
!pip install pulp

o con !pip dentro de Jupyter Notebook:

In [ ]:
!pip install pulp

Esto asegurará que la librería esté disponible para su uso en el código.

**Carga de Datos**

Se lee el archivo CSV con los datos de distribución de ejemplares.

In [ ]:
df = pd.read_csv(ruta_archivo)

**Definición del Modelo de Optimización**

In [ ]:
modelo = LpProblem("Optimización_Distribución", LpMaximize)

Se define un modelo de programación lineal con el objetivo de maximizar las ventas (Vendido_Optimizado).

Definición de Variables de Decisión

In [ ]:
envio_total = { (row.dia_semana, row.idsector): LpVariable(f"envio_{row.dia_semana}_{row.idsector}", lowBound=row.envio_total, cat='Integer')
                for _, row in df.iterrows() }


*   Envio_Optimizado: Cantidad de ejemplares enviados a cada sector por día.
*   Promo_Optimizado: Cantidad destinada a promociones (máximo 15 unidades)
*   Flete_Optimizado: Cantidad destinada a envíos (máximo 30 unidades).
*   Devuelto_Optimizado: Cantidad de ejemplares devueltos (calculada dinámicamente).

**Cálculo del Factor de Devolución**

In [ ]:
factor_devolucion = { (row.dia_semana, row.idsector): row.devuelto / row.envio_total if row.envio_total > 0 else 0
                      for _, row in df.iterrows() }

Se calcula una tasa histórica de devolución para cada sector y día.

**Función Objetivo: Maximizar Ventas**

In [ ]:
modelo += lpSum((envio_total[i] - promo[i] - flete[i] - devuelto_opt[i]) for i in envio_total), "Maximizar_Ventas"

Maximiza la cantidad de unidades vendidas (Vendido_Optimizado), tomando en cuenta las devoluciones, las promociones y los costos de flete.

**Restricciones del Modelo**

In [ ]:
# No superar el total de ejemplares disponibles por día
for dia in df["dia_semana"].unique():
    modelo += lpSum(envio_total[i] for i in envio_total if i[0] == dia) <= total_ejemplares_por_dia[dia], f"Restriccion_Total_Disponible_{dia}"

Se asegura que la cantidad de envíos no supere la capacidad máxima de producción diaria.

In [ ]:
# La sumatoria de unidades vendidas no puede superar el pronóstico diario
for dia in df["dia_semana"].unique():
    modelo += lpSum(envio_total[i] - promo[i] - flete[i] - devuelto_opt[i] for i in envio_total if i[0] == dia) <= pronostico_ventas_por_dia[dia], f"Restriccion_Pronostico_Ventas_{dia}"

Asegura que las ventas no sobrepasen el pronóstico de demanda diario.

In [ ]:
# La cantidad enviada debe ser al menos igual a la cantidad vendida históricamente
for (dia, sector) in envio_total:
    modelo += envio_total[(dia, sector)] >= df.set_index(['dia_semana', 'idsector']).loc[(dia, sector), 'vendido'], f"Restriccion_Minimo_Envio_{dia}_{sector}"

Evita que Envio_Optimizado sea menor que la cantidad vendida históricamente.

In [ ]:
# Evitar valores negativos en la cantidad vendida
for (dia, sector) in envio_total:
    modelo += envio_total[(dia, sector)] - promo[(dia, sector)] - flete[(dia, sector)] - devuelto_opt[(dia, sector)] >= 0, f"Restriccion_No_Negativo_Ventas_{dia}_{sector}"

Asegura que Vendido_Optimizado nunca sea negativo.

In [ ]:
# Ajustar la cantidad de devoluciones en función de la cantidad enviada
for (dia, sector) in devuelto_opt:
    modelo += devuelto_opt[(dia, sector)] == envio_total[(dia, sector)] * factor_devolucion[(dia, sector)], f"Restriccion_Devolucion_Segun_Envio_{dia}_{sector}"

Calcula Devuelto_Optimizado como un porcentaje dinámico de Envio_Optimizado.

**Resolviendo el Problema**

In [ ]:
modelo.solve()

Se ejecuta el solver de PuLP para encontrar la mejor solución.

**Guardando los Resultados**

In [ ]:
df["Envio_Optimizado"] = [envio_total[(row.dia_semana, row.idsector)].varValue for _, row in df.iterrows()]
df["Promo_Optimizado"] = [promo[(row.dia_semana, row.idsector)].varValue for _, row in df.iterrows()]
df["Flete_Optimizado"] = [flete[(row.dia_semana, row.idsector)].varValue for _, row in df.iterrows()]
df["Devuelto_Optimizado"] = [devuelto_opt[(row.dia_semana, row.idsector)].varValue for _, row in df.iterrows()]
df["Vendido_Optimizado"] = df["Envio_Optimizado"] - df["Promo_Optimizado"] - df["Flete_Optimizado"] - df["Devuelto_Optimizado"]

Se almacenan los valores optimizados en el dataframe.

In [ ]:
df.to_csv("solucion_optimizada_sectores.csv", index=False)

Los resultados se guardan en un archivo CSV para su análisis posterior.

R**esumen del Pipeline**

1. Se cargan los datos históricos de distribución y ventas.
2. Se definen las variables de decisión (Envio_Optimizado, Promo_Optimizado, Flete_Optimizado, Devuelto_Optimizado).
3. Se establece la función objetivo para maximizar Vendido_Optimizado y minimizar Devuelto_Optimizado.
4. Se aplican restricciones de capacidad, demanda y valores mínimos.
5. El modelo se resuelve utilizando PuLP y los resultados se almacenan en un CSV.

# **Código de Optimización de Distribución de Ejemplares por Municipio - PuLP**

In [ ]:
!pip install pulp
!pip install pulp


In [ ]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

def optimizar_distribucion(ruta_archivo, total_ejemplares_por_dia, pronostico_ventas_por_dia):
    try:
        # Leer el archivo CSV
        df = pd.read_csv(ruta_archivo)

        # Crear el problema de optimización
        modelo = LpProblem("Optimización_Distribución", LpMaximize)

        # Crear variables de decisión para determinar promo, flete y envio_total por municpio y día de la semana
        envio_total = { (row.dia_semana, row.idmupiogeo): LpVariable(f"envio_{row.dia_semana}_{row.idmupiogeo}", lowBound=row.envio_total, cat='Integer')
                        for _, row in df.iterrows() }

        promo = { (row.dia_semana, row.idmupiogeo): LpVariable(f"promo_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, upBound=154, cat='Integer')
                  for _, row in df.iterrows() }

        flete = { (row.dia_semana, row.idmupiogeo): LpVariable(f"flete_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, upBound=1324, cat='Integer')
                  for _, row in df.iterrows() }

        devuelto_opt = { (row.dia_semana, row.idmupiogeo): LpVariable(f"devuelto_{row.dia_semana}_{row.idmupiogeo}", lowBound=0, cat='Integer')
                         for _, row in df.iterrows() }

        # Calcular un factor de devolución basado en datos históricos
        factor_devolucion = { (row.dia_semana, row.idmupiogeo): row.devuelto / row.envio_total if row.envio_total > 0 else 0
                              for _, row in df.iterrows() }

        # Función objetivo: Maximizar ventas y minimizar devoluciones
        modelo += lpSum((envio_total[i] - promo[i] - flete[i] - devuelto_opt[i]) for i in envio_total), "Maximizar_Ventas"

        # Restricción: No superar el total de ejemplares disponibles a nivel nacional por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] for i in envio_total if i[0] == dia) <= total_ejemplares_por_dia[dia], f"Restriccion_Total_Disponible_{dia}"

        # Restricción: La sumatoria de unidades vendidas no puede ser mayor al pronóstico de ventas por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] - promo[i] - flete[i] - devuelto_opt[i] for i in envio_total if i[0] == dia) <= pronostico_ventas_por_dia[dia], f"Restriccion_Pronostico_Ventas_{dia}"

        # Restricción: Asegurar que envio_total sea mayor o igual que vendido
        for (dia, municipio) in envio_total:
            modelo += envio_total[(dia, municipio)] >= df.set_index(['dia_semana', 'idmupiogeo']).loc[(dia, municipio), 'vendido'], f"Restriccion_Minimo_Envio_{dia}_{municipio}"

        # Restricción: Evitar valores negativos en Vendido_Optimizado
        for (dia, municipio) in envio_total:
            modelo += envio_total[(dia, municipio)] - promo[(dia, municipio)] - flete[(dia, municipio)] - devuelto_opt[(dia, municipio)] >= 0, f"Restriccion_No_Negativo_Ventas_{dia}_{municipio}"

        # Restricción: Ajustar Devuelto_Optimizado basado en Envio_Optimizado
        for (dia, municipio) in devuelto_opt:
            modelo += devuelto_opt[(dia, municipio)] == envio_total[(dia, municipio)] * factor_devolucion[(dia, municipio)], f"Restriccion_Devolucion_Segun_Envio_{dia}_{municipio}"

        # Resolver el problema
        modelo.solve()

        # Guardar los resultados en el DataFrame
        df["Envio_Optimizado"] = [envio_total[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Promo_Optimizado"] = [promo[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Flete_Optimizado"] = [flete[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Devuelto_Optimizado"] = [devuelto_opt[(row.dia_semana, row.idmupiogeo)].varValue for _, row in df.iterrows()]
        df["Vendido_Optimizado"] = df["Envio_Optimizado"] - df["Promo_Optimizado"] - df["Flete_Optimizado"] - df["Devuelto_Optimizado"]

        # Guardar la solución en un CSV
        df.to_csv("solucion_optimizada_municipio.csv", index=False)
        print("Optimización completada. Resultados guardados en 'solucion_optimizada_municipio.csv'")

    except Exception as e:
        print(f"Error al ejecutar la optimización: {e}")

# Ruta del archivo CSV para sectores
ruta_csv = "/content/datos_finales_municipio.csv"  # Reemplaza con la ruta real

# Diccionarios con valores de total de ejemplares y pronóstico por día de la semana
total_ejemplares_por_dia = {
    "Lunes": 75000,
    "Martes": 75000,
    "Miercoles": 75000,
    "Jueves": 75000,
    "Viernes": 75000,
    "Sabado": 75000,
    "Domingo": 75000
}

pronostico_ventas_por_dia = {
    "Lunes": 68000,
    "Martes": 65000,
    "Miercoles": 61000,
    "Jueves": 68000,
    "Viernes": 63000,
    "Sabado": 63000,
    "Domingo": 69000
}

optimizar_distribucion(ruta_csv, total_ejemplares_por_dia, pronostico_ventas_por_dia)

________________________________________________________________________________________________________________________________________________________________

# **Código de Optimización de Distribución de Ejemplares por Regional - PuLP**

In [6]:
!pip install pulp
!pip install pulp

In [7]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

def optimizar_distribucion(ruta_archivo, total_ejemplares_por_dia, pronostico_ventas_por_dia):
    try:
        # Leer el archivo CSV
        df = pd.read_csv(ruta_archivo)

        # Crear el problema de optimización
        modelo = LpProblem("Optimización_Distribución", LpMaximize)

        # Crear variables de decisión para determinar promo, flete y envio_total por regional y día de la semana
        envio_total = { (row.dia_semana, row.idregional): LpVariable(f"envio_{row.dia_semana}_{row.idregional}", lowBound=row.envio_total, cat='Integer')
                        for _, row in df.iterrows() }

        promo = { (row.dia_semana, row.idregional): LpVariable(f"promo_{row.dia_semana}_{row.idregional}", lowBound=0, upBound=185, cat='Integer')
                  for _, row in df.iterrows() }

        flete = { (row.dia_semana, row.idregional): LpVariable(f"flete_{row.dia_semana}_{row.idregional}", lowBound=0, upBound=2088, cat='Integer')
                  for _, row in df.iterrows() }

        devuelto_opt = { (row.dia_semana, row.idregional): LpVariable(f"devuelto_{row.dia_semana}_{row.idregional}", lowBound=0, cat='Integer')
                         for _, row in df.iterrows() }

        # Calcular un factor de devolución basado en datos históricos
        factor_devolucion = { (row.dia_semana, row.idregional): row.devuelto / row.envio_total if row.envio_total > 0 else 0
                              for _, row in df.iterrows() }

        # Función objetivo: Maximizar ventas y minimizar devoluciones
        modelo += lpSum((envio_total[i] - promo[i] - flete[i] - devuelto_opt[i]) for i in envio_total), "Maximizar_Ventas"

        # Restricción: No superar el total de ejemplares disponibles a nivel nacional por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] for i in envio_total if i[0] == dia) <= total_ejemplares_por_dia[dia], f"Restriccion_Total_Disponible_{dia}"

        # Restricción: La sumatoria de unidades vendidas no puede ser mayor al pronóstico de ventas por día
        for dia in df["dia_semana"].unique():
            modelo += lpSum(envio_total[i] - promo[i] - flete[i] - devuelto_opt[i] for i in envio_total if i[0] == dia) <= pronostico_ventas_por_dia[dia], f"Restriccion_Pronostico_Ventas_{dia}"

        # Restricción: Asegurar que envio_total sea mayor o igual que vendido
        for (dia, regional) in envio_total:
            modelo += envio_total[(dia, regional)] >= df.set_index(['dia_semana', 'idregional']).loc[(dia, regional), 'vendido'], f"Restriccion_Minimo_Envio_{dia}_{regional}"

        # Restricción: Evitar valores negativos en Vendido_Optimizado
        for (dia, regional) in envio_total:
            modelo += envio_total[(dia, regional)] - promo[(dia, regional)] - flete[(dia, regional)] - devuelto_opt[(dia, regional)] >= 0, f"Restriccion_No_Negativo_Ventas_{dia}_{regional}"

        # Restricción: Ajustar Devuelto_Optimizado basado en Envio_Optimizado
        for (dia, regional) in devuelto_opt:
            modelo += devuelto_opt[(dia, regional)] == envio_total[(dia, regional)] * factor_devolucion[(dia, regional)], f"Restriccion_Devolucion_Segun_Envio_{dia}_{regional}"

        # Resolver el problema
        modelo.solve()

        # Guardar los resultados en el DataFrame
        df["Envio_Optimizado"] = [envio_total[(row.dia_semana, row.idregional)].varValue for _, row in df.iterrows()]
        df["Promo_Optimizado"] = [promo[(row.dia_semana, row.idregional)].varValue for _, row in df.iterrows()]
        df["Flete_Optimizado"] = [flete[(row.dia_semana, row.idregional)].varValue for _, row in df.iterrows()]
        df["Devuelto_Optimizado"] = [devuelto_opt[(row.dia_semana, row.idregional)].varValue for _, row in df.iterrows()]
        df["Vendido_Optimizado"] = df["Envio_Optimizado"] - df["Promo_Optimizado"] - df["Flete_Optimizado"] - df["Devuelto_Optimizado"]

        # Guardar la solución en un CSV
        df.to_csv("solucion_optimizada_regional.csv", index=False)
        print("Optimización completada. Resultados guardados en 'solucion_optimizada_regional.csv'")

    except Exception as e:
        print(f"Error al ejecutar la optimización: {e}")

# Ruta del archivo CSV para sectores
ruta_csv = "/content/datos_finales_regional.csv"  # Reemplaza con la ruta real

# Diccionarios con valores de total de ejemplares y pronóstico por día de la semana
total_ejemplares_por_dia = {
    "Lunes": 75000,
    "Martes": 75000,
    "Miercoles": 75000,
    "Jueves": 75000,
    "Viernes": 75000,
    "Sabado": 75000,
    "Domingo": 75000
}

pronostico_ventas_por_dia = {
    "Lunes": 68000,
    "Martes": 65000,
    "Miercoles": 61000,
    "Jueves": 68000,
    "Viernes": 63000,
    "Sabado": 63000,
    "Domingo": 69000
}

optimizar_distribucion(ruta_csv, total_ejemplares_por_dia, pronostico_ventas_por_dia)

Optimización completada. Resultados guardados en 'solucion_optimizada_regional.csv'


________________________________________________________________________________________________________________________________________________________________